# Pretrained DenseNet161 PyTorch Baseline Train (5-fold)


Newish to Kaggle, and this is my first ever notebook! 

I simply tried an ImageNet pretrained Densenet161 for **training** with 5 fold cross validation.  It appears this is not good enough (well in comparison to the other notebooks with much higher performances). I haven't added any augmentations and are resizing my images to 224 by 224 (see the dataset class). 

This notebook is for simplicity and so I hope it is useful for those who want a bareback simple example to build on. I'm also big on ockhams razor so don't want to add complexity where it is not needed.

For future work, I will:
* resize images to 512 by 512 pixels, 
* use an efficientnet architecture or se_resnext101 
* add simple augmentations
* a different loss function as it is also understood that there may be a lot of noise in the data (i.e. noisy labels).

Let me know what you think. 

In [ ]:
# import libraries 

import torch
import torchvision.models as models
import numpy as np
import pandas as pd
import random
import os
import pandas as pd
import cv2

import PIL
from PIL import Image

from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets
from torchvision import models as models
from torchvision import transforms
from torch.autograd import Variable
from torchvision import transforms as T


torch.backends.cudnn.deterministic = True

import albumentations
from sklearn import metrics, model_selection


pathtoimgs = "../input/cassava-leaf-disease-classification/train_images"  # Path to folder with train images
pathtocsv = "../input/cassava-leaf-disease-classification/train.csv"  # Path to csv-file with targets
path = ""  # Working directory, the place where the logs and the weights will be saved

# define parameters etc.

n_folds = 5
seed = 33
scheduler_step = 20 
scheduler_gamma = 5
lr = 0.0001
batch_size = 32
num_classes = 5
num_epochs = 5
num_workers = 4

In [ ]:
# create folds taken from https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-inference-tta
df = pd.read_csv(pathtocsv)
df["kfold"] = -1    
df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=n_folds, random_state = seed, shuffle=True)

for f, (t_, v_) in enumerate(kf.split(np.arange(df.shape[0]), df.label.values)):
    df.loc[df.index.isin(v_), ['kfold']] = f
    
df.to_csv("train_folds.csv", index=False)

N_CLASSES = df.label.nunique()
LABELS = ['Cassava Bacterial Blight','Cassava Brown Streak Disease','Cassava Green Mottle','Cassava Mosaic Disease','Healthy']

Defining the Dataset class. I optdd for something quite simple. You can easily add 

In [ ]:

class ClassificationDataset:
    def __init__(self, image_paths, targets):
        self.image_paths = image_paths
        self.targets = targets        
        self.transforms = T.Compose([T.RandomResizedCrop(224),T.ToTensor(),
                                T.Normalize([0.670, 0.464, 0.707], [0.146, 0.165, 0.111])]) # ImageNet values

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item]).convert('RGB')
        image = self.transforms(image)
        targets = self.targets[item]
    
        return image, targets 



Here I define the pretrained DenseNet161 feature extractor.

In [ ]:
class DenseNetConv(torch.nn.Module): # https://www.kaggle.com/renatobmlr/pytorch-densenet-as-feature-extractor
    def __init__(self):
        super(DenseNetConv,self).__init__()
        original_model = models.densenet161(pretrained=True)
        self.features = torch.nn.Sequential(*list(original_model.children())[:-1])
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.features(x)
        #x = F.relu(x, inplace=True)
        x = F.avg_pool2d(x, kernel_size=7).view(x.size(0), -1)
        return x
    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

densenet= DenseNetConv()
densenet.to(device)
print(densenet)

The fully connected layer/classifier is defined below. The DenseNet feature extractor should output 2208 features thus we define a linear layer that contains the number of features and classes.

In [ ]:
classifier = nn.Linear(2208, num_classes)
classifier.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)


In [ ]:
for fold in range(n_folds):
    
    training_data_path = pathtoimgs
    df = pd.read_csv("train_folds.csv")

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_images = df_train.image_id.values.tolist()
    train_images = [os.path.join(training_data_path, i ) for i in train_images]
    train_targets = df_train.label.values

    valid_images = df_valid.image_id.values.tolist()
    valid_images = [os.path.join(training_data_path, i ) for i in valid_images]
    valid_targets = df_valid.label.values
    
    
    train_dict = ClassificationDataset(image_paths=train_images, targets=train_targets)    
    train_loader = torch.utils.data.DataLoader(train_dict, batch_size=batch_size, shuffle=True, num_workers=num_workers) 

    val_dict = ClassificationDataset(image_paths=valid_images, targets=valid_targets)    
    val_loader = torch.utils.data.DataLoader(val_dict, batch_size=batch_size, shuffle=True, num_workers=num_workers) 

   

    for epoch in range(0, num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 60)

        total_loss = 0
        train_accuracy = 0
        classifier.train()

        for i, (inputs, labels) in enumerate(train_loader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            

            x10 = densenet(inputs)
        

            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs =  classifier(x10)
           
            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, pred = torch.max(outputs, 1)
            equality_check = (labels.data == pred)
            train_accuracy += equality_check.type(torch.FloatTensor).mean()

            # Getting gradients w.r.t. parameters
            loss.backward()
            optimizer.step()

        scheduler.step()

        train_losses = total_loss / len(train_loader)
        training_accuracy = train_accuracy/ len(train_loader)

        print("Epoch : {} Train Loss = {:.6f}, Train Accuracy = {:.6f}".format(epoch, train_losses, training_accuracy))

        
        classifier.eval()
        val_loss = 0
        val_accuracy = 0

        val_confusion_matrix = torch.zeros(num_classes, num_classes)

        with torch.no_grad(): # Tell torch not to calculate gradients
            for i, (inputs, labels) in enumerate(val_loader):
                val_inputs10x = inputs.to(device)
                val_labels = labels.to(device)

                val_x10 = densenet(val_inputs10x)

                val_outputs = classifier(val_x10)
                val_loss += criterion(val_outputs, val_labels).item()

                _val, val_pred = torch.max(val_outputs, 1)
                val_equality_check = (val_labels.data == val_pred)
                val_accuracy += val_equality_check.type(torch.FloatTensor).mean()

                for t, p in zip(val_labels.view(-1), val_pred.view(-1)):
                    val_confusion_matrix[t.long(), p.long()] += 1
                    
        # https://discuss.pytorch.org/t/class-wise-accuacy/88141/2
        ele_wise_acc = val_confusion_matrix.diag() / val_confusion_matrix.sum(1)
        class_wise_acc = ele_wise_acc.mean() * 100


        # https://stackoverflow.com/questions/52176178/pytorch-model-accuracy-test

        val_losses = val_loss / len(val_loader)
        validation_accuracy = val_accuracy/ len(val_loader)


        print("Epoch : {} Validation Loss = {:.6f}, Validation Accuracy = {:.6f}".format(epoch, val_losses, validation_accuracy))
